In [6]:
import numpy as np

T, H = 5, 4
hs = np.random.randn(T, H)
print(hs.shape)
a  = np.array([0.8, 0.1, 0.03, 0.05, 0.02])
print(a)
ar = a.reshape(T, 1)
print(ar)
ar = ar.repeat(4, axis=1)
print(ar.shape)
print(ar)

t = hs * ar
print(t.shape)

c = np.sum(t, axis=0)
print(c.shape)
print(c)

(5, 4)
[0.8  0.1  0.03 0.05 0.02]
[[0.8 ]
 [0.1 ]
 [0.03]
 [0.05]
 [0.02]]
(5, 4)
[[0.8  0.8  0.8  0.8 ]
 [0.1  0.1  0.1  0.1 ]
 [0.03 0.03 0.03 0.03]
 [0.05 0.05 0.05 0.05]
 [0.02 0.02 0.02 0.02]]
(5, 4)
(4,)
[ 0.26671291 -1.7432512   1.12530281 -0.13507866]


In [8]:
import numpy as np

N, T, H = 2, 3, 4
hs = np.random.randn(N, T, H)
a  = np.random.randn(N, T)

ar = a.reshape(N, T, 1).repeat(H, axis=2)
print(ar.shape)

t = hs * ar
print(t.shape)

c = np.sum(t, axis=1)
print(c.shape)

(2, 3, 4)
(2, 3, 4)
(2, 4)


In [9]:
t = np.arange(24).reshape(2, 3, 4)
print(t)
c = np.sum(t, axis=1)
print(c)

[[[ 0  1  2  3]
  [ 4  5  6  7]
  [ 8  9 10 11]]

 [[12 13 14 15]
  [16 17 18 19]
  [20 21 22 23]]]
[[12 15 18 21]
 [48 51 54 57]]


In [10]:
import numpy as np

hs = np.array([[1,1,1,1],
               [1,1,-1,-1],
               [-1,-1,1,1],
               [-1,-1,-1,-1]])

h = np.array([1,1,-1,-1])

for  h_temp in hs:
    print(np.dot(h_temp, h))

0
4
-4
0


In [14]:
import sys
sys.path.append("..")
from common.layers import Softmax
import numpy as np

N, T, H = 10, 5, 4
hs = np.random.randn(N, T, H)
h  = np.random.randn(N, H)
print(h.shape)

hr = h.reshape(N, 1, H).repeat(T, axis=1)
print(hr.shape)

t = hs * hr
print(t.shape)

s = np.sum(t, axis=2)
print(s.shape)
print(s)

softmax = Softmax()
a = softmax.forward(s)
print(a.shape)
print(a)

(10, 4)
(10, 5, 4)
(10, 5, 4)
(10, 5)
[[ 2.13895413  0.16180462  2.05041054 -0.46732199  1.58900983]
 [ 0.65173633 -1.73086554  2.05438564  0.81005208 -1.53265012]
 [-1.32132097  2.20900552 -1.39824675 -2.65334664 -0.30308086]
 [ 0.60162264 -0.17431357 -0.65476625  1.57101249 -0.81945623]
 [-0.03013397 -0.19250297  0.0483796  -0.016538   -0.81958487]
 [ 1.62560246  0.09045867  0.39314173  1.68972043 -0.34632695]
 [-0.87209962  0.12861409 -0.0237351   1.11858666  0.355926  ]
 [-0.62680912 -3.45963901 -2.00985498 -0.19035552  4.36741573]
 [-1.6620005  -0.70177452 -1.55369295  1.89976814  1.24906882]
 [-0.74218738 -0.45686661 -0.1957335   0.17768398  1.81482014]]
(10, 5)
[[3.69751732e-01 5.11970688e-02 3.38420160e-01 2.72909642e-02
  2.13340075e-01]
 [1.55222846e-01 1.43286343e-02 6.31129530e-01 1.81849160e-01
  1.74698293e-02]
 [2.55798113e-02 8.73169689e-01 2.36858456e-02 6.75158807e-03
  7.08130659e-02]
 [2.16321845e-01 9.95670328e-02 6.15825427e-02 5.70296939e-01
  5.22316407e-02]
 [2

In [ ]:
# coding: utf-8
import sys
sys.path.append('..')
from common.np import *  # import numpy as np
from common.layers import Softmax


class WeightSum:
    def __init__(self):
        self.params, self.grads = [], []
        self.cache = None

    def forward(self, hs, a):
        N, T, H = hs.shape

        ar = a.reshape(N, T, 1).repeat(H, axis=2)
        t = hs * ar
        c = np.sum(t, axis=1)

        self.cache = (hs, ar)
        return c

    def backward(self, dc):
        hs, ar = self.cache
        N, T, H = hs.shape
        dt = dc.reshape(N, 1, H).repeat(T, axis=1)
        dar = dt * hs
        dhs = dt * ar
        da = np.sum(dar, axis=2)

        return dhs, da


class AttentionWeight:
    def __init__(self):
        self.params, self.grads = [], []
        self.softmax = Softmax()
        self.cache = None

    def forward(self, hs, h):
        N, T, H = hs.shape

        hr = h.reshape(N, 1, H).repeat(T, axis=1)
        t = hs * hr
        s = np.sum(t, axis=2)
        a = self.softmax.forward(s)

        self.cache = (hs, hr)
        return a

    def backward(self, da):
        hs, hr = self.cache
        N, T, H = hs.shape

        ds = self.softmax.backward(da)
        dt = ds.reshape(N, T, 1).repeat(H, axis=2)
        dhs = dt * hr
        dhr = dt * hs
        dh = np.sum(dhr, axis=1)

        return dhs, dh


class Attention:
    def __init__(self):
        self.params, self.grads = [], []
        self.attention_weight_layer = AttentionWeight()
        self.weight_sum_layer = WeightSum()
        self.attention_weight = None

    def forward(self, hs, h):
        a = self.attention_weight_layer.forward(hs, h)
        out = self.weight_sum_layer.forward(hs, a)
        self.attention_weight = a
        return out

    def backward(self, dout):
        dhs0, da = self.weight_sum_layer.backward(dout)
        dhs1, dh = self.attention_weight_layer.backward(da)
        dhs = dhs0 + dhs1
        return dhs, dh


class TimeAttention:
    def __init__(self):
        self.params, self.grads = [], []
        self.layers = None
        self.attention_weights = None

    def forward(self, hs_enc, hs_dec):
        N, T, H = hs_dec.shape
        out = np.empty_like(hs_dec)
        self.layers = []
        self.attention_weights = []

        for t in range(T):
            layer = Attention()
            out[:, t, :] = layer.forward(hs_enc, hs_dec[:,t,:])
            self.layers.append(layer)
            self.attention_weights.append(layer.attention_weight)

        return out

    def backward(self, dout):
        N, T, H = dout.shape
        dhs_enc = 0
        dhs_dec = np.empty_like(dout)

        for t in range(T):
            layer = self.layers[t]
            dhs, dh = layer.backward(dout[:, t, :])
            dhs_enc += dhs
            dhs_dec[:,t,:] = dh

        return dhs_enc, dhs_dec
